In [1]:
#Importing libraries
import cv2
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def make_coordinates(image, line_parameters):
    try:
        slope, intercept = line_parameters
    except TypeError:
        slope, intercept = 0.001, 0 
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array([x1, y1, x2, y2])

In [3]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            par= np.polyfit((x1,x2), (y1,y2), 1)
            slope = par[0]
            intercept = par[1]
            if  slope < 0:
                left_fit.append((slope, intercept))
            else:
                right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])

In [4]:
def Canny(image):
    gray= cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    blur= cv2.GaussianBlur(gray, (5,5), 0)
    canny= cv2.Canny(blur,50, 150)
    return canny

In [5]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1,y1,x2,y2 in lines:
            cv2.line(line_image, (x1,y1), (x2,y2), (255,0,0),10)
    return line_image

In [6]:
def region_of_intrest(image):
    polygons = np.array([
    [(150,850), (1600, 850), (950, 450)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [7]:
#Object Detection
thres = 0.50
classNames = []
classFile = 'coco.names'
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 000001D45443DD50>

In [13]:
#importing video
cap= cv2.VideoCapture("car_cam.mp4")
while True:
    _, frame = cap.read()
    frame = cv2.resize(frame,(1920,1080))
    lane_image = np.copy(frame)
    canny = Canny(lane_image)
    cropped_image = region_of_intrest(canny)
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 140, np.array([]), minLineLength=10, maxLineGap=100 )
    averaged_lines=average_slope_intercept(lane_image,lines)
    line_image=display_lines(lane_image, averaged_lines)
    combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)
    classIds, confs, bbox = net.detect(combo_image,confThreshold=thres)
    print(classIds,bbox)
    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(),confs.flatten(),bbox):
            if(classId == 2 or classId == 3 or classId == 10 or classId == 6 or  classId == 9 or classId == 11 ):
                cv2.rectangle(combo_image,box,color=(0,255,0),thickness=2)
                cv2.putText(combo_image,classNames[classId-1].upper(),(box[0]+10,box[1]+30),
                        cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                cv2.putText(combo_image,str(round(confidence*100,2)),(box[0]+200,box[1]+30),
                        cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    cv2.imshow("Output",combo_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[[38]] [[   5  419 1797  661]]
() ()
() ()
() ()
() ()
[[3]] [[1841  545   54   24]]
() ()
() ()
() ()
() ()
() ()
() ()
() ()
[[28]] [[   0  411 1794  665]]
[[28]] [[   0  416 1810  661]]
[[ 1]
 [28]] [[1461  535   55  142]
 [   3  419 1780  659]]
[[1]] [[1481  529   52  163]]
() ()
[[38]] [[   8  420 1833  660]]
[[38]] [[   4  423 1819  657]]
[[38]] [[  22  406 1860  669]]
[[38]] [[  25  412 1849  662]]
[[28]
 [38]] [[   2  421 1782  655]
 [   4  424 1794  656]]
() ()
[[38]] [[  13  427 1733  652]]
[[38]] [[  13  428 1755  651]]
[[38]] [[  11  426 1781  653]]
[[38]] [[  15  426 1752  653]]
[[38]] [[  12  421 1831  659]]
[[38]] [[  10  421 1818  658]]
[[38]] [[  15  424 1797  655]]
[[38]] [[   8  424 1797  655]]
[[38]] [[  11  428 1824  651]]
[[38]] [[  11  424 1829  656]]
() ()
[[38]] [[  11  425 1726  653]]
[[28]
 [38]] [[   0  420 1782  655]
 [   9  427 1750  651]]
[[28]
 [38]] [[   0  418 1777  658]
 [   9  429 1742  649]]
[[28]
 [38]] [[   0  415 1793  663]
 [  10  423 1746  656]

() ()
[[28]] [[   0  427 1846  649]]
() ()
() ()
() ()
() ()
() ()
() ()
[[3]] [[629 382  61  30]]
() ()
() ()
() ()
() ()
[[28]] [[   9  427 1889  651]]
[[ 5]
 [28]] [[   3  430 1828  650]
 [  11  425 1862  654]]
[[5]] [[   9  429 1837  651]]
() ()
[[5]] [[   0  433 1828  647]]
[[5]] [[   8  429 1861  651]]
[[5]] [[   8  431 1811  649]]
() ()
() ()
[[3]] [[   0  421 1870  659]]
[[5]] [[   0  433 1872  643]]
[[5]] [[  17  409 1869  664]]
[[5]] [[  10  431 1825  649]]
[[5]] [[  12  432 1822  648]]
[[5]] [[   0  425 1849  652]]
() ()
() ()
() ()
[[28]] [[   0  418 1822  658]]
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
() ()
[[3]] [[1811  501  105  134]]
[[3]] [[1826  505   86  142]]
[[3]
 [3]] [[354 425 101  71]
 [317 433  89  70]]
() ()
() ()
[[3]] [[1450  482   97   65]]
[[3]] [[256 431 135  80]]
[[3]] [[1473  471  173   88]]
[[3]] [[1476  465  200  107]]
[[3]] [

[[10]
 [10]
 [28]] [[ 997  326   81   87]
 [ 987  346   73   79]
 [   0  409 1892  668]]
[[28]] [[  11  415 1861  663]]
[[10]
 [10]
 [28]] [[ 998  324   86   90]
 [ 989  344   83   83]
 [  15  406 1844  672]]
[[10]
 [10]
 [28]] [[ 991  318   96   99]
 [ 982  344   90   83]
 [  22  407 1851  672]]
[[10]
 [28]] [[ 979  316  117  101]
 [  14  408 1855  671]]
[[28]] [[  13  416 1853  664]]
[[28]] [[   9  418 1867  662]]
[[28]] [[   8  412 1833  668]]
[[28]] [[  15  421 1864  659]]
[[28]] [[   4  409 1829  671]]
[[28]] [[   0  417 1842  663]]
[[28]] [[   0  414 1815  666]]
[[28]] [[   0  407 1783  673]]
[[28]] [[   8  415 1826  665]]
() ()
() ()
[[28]] [[  16  370 1892  707]]
[[28]] [[  16  374 1890  704]]
[[ 3]
 [28]] [[   8  414 1852  666]
 [   8  401 1878  679]]
[[ 3]
 [28]] [[  16  377 1890  703]
 [   4  408 1881  672]]
[[28]] [[  16  350 1891  730]]
[[3]] [[  16  363 1892  717]]
[[3]] [[  17  362 1885  718]]
[[1]
 [3]] [[1056  315  144  159]
 [  18  359 1889  718]]
[[3]] [[  18  356 18